# Mask R-CNN

In [ ]:
import os
import cv2 
import sys
import random
import math
import skimage.io
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from flask import Flask, request, Response, render_template, send_file

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# COCO trained weights
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

## Configurations 

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Object detection class

In [ ]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
# make it alphabetical order
# class_names.sort()
# print(class_names)

## Image anotation functions

In [ ]:
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

# todo: list solution, for user/web choices.

def display_instances(image, boxes, masks, ids, names, choices, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue
        # if names[ids[i]] != choice:
        #     continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        if label not in choices:
            continue
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)

        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image

## Flask app

In [ ]:
#  app initialization
app = Flask(__name__)
# This route will serve the index.html page
@app.route('/')
def index():
    return render_template('index.html')

# This route will process the video and perform object detection on chosen classes
@app.route('/process_video', methods=['POST'])
def process_video():
    with tf.Session() as sess:
        # Load a (frozen) model into memory.
        sess.run(tf.global_variables_initializer())

        with sess.as_default():
            model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

        # Load weights trained on MS-COCO
        model.load_weights(COCO_MODEL_PATH, by_name=True)

        video = request.files["video"]
        video.save("video.mp4")

        selected_classes = request.form.getlist("classes")
        print(f"selected_classes: {selected_classes}")
        if not selected_classes:
            print("Error: class names list is empty")
        
        class_names_selected = [class_name for class_name in class_names if class_name in selected_classes]
        # indexed_classes = [class_names.index(class_name) for class_name in class_names_selected]
        # print(f"indexed_classes: {indexed_classes}")
        # print(f"class_names_selected: {class_names_selected}")
        
        # open video file
        capture = cv2.VideoCapture("video.mp4")

        # the 2 lines can be removed if you dont have a 1080p camera.
        capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
        capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

        # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
        frame_width = int(capture.get(3))
        frame_height = int(capture.get(4))
        out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

        # loop through frames
        while True:
            ret, frame = capture.read()
            if not ret:
                print("Can't receive frame (stream end?). Exiting ..." )
                break
            
            # Run detection
            results = model.detect([frame], verbose=0)
            r = results[0]
            # print(r)
            frame = display_instances(
                frame, r['rois'], r['masks'], r['class_ids'], class_names, class_names_selected, r['scores']
            )
            cv2.imshow('frame', frame)
            out.write(frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        capture.release()
        cv2.destroyAllWindows()
    return send_file("output.avi", as_attachment=True)

# if __name__ == '__main__':
app.run(debug=True, use_reloader=False)